# ***BEST TEST ERRORS WITHIN THIS NOTEBOOK (ORDERED BY ACCURACY)***

> Neural Network Model: 0.2275 (See BEST NN MODEL FOUND Section)
---
> KNN Classifier Model: 0.2502
---
> Linear Regression Model: 0.3089
---
> Ridge Regression Model: 0.3089
---
> Lasso Regression Model: 0.9816
---
> Elastic Net CV  Model: 0.9858
---






# Importing Libraries

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Backward-Stepwise Variable Selection with AIC

In [ ]:
ATM_train = pd.read_csv("ATM_training.csv")
ATM_train = ATM_train.dropna()

In [ ]:
aics = {'None':[],'Shops':[], 'ATMs':[], 'Downtown':[], 'Weekday':[], 'Center':[],'High':[]}

formula = 'Withdraw ~ Shops + ATMs + Downtown + Weekday + Center + High'
model = smf.ols(formula, ATM_train)
results = model.fit()
aics["None"].append(results.aic)

for predictor in ['Shops', 'ATMs', 'Downtown', 'Weekday', 'Center','High']:
  array = ['Shops', 'ATMs', 'Downtown', 'Weekday', 'Center','High']
  formula = 'Withdraw ~ '
  array.remove(predictor)
  count = 0
  for pred in array:
    if count == 0:
      formula += ' ' + pred
      count += 1
    else:
      formula += ' + ' + pred
  model = smf.ols(formula, ATM_train)
  results = model.fit()
  aics[predictor].append(results.aic)
for val in sorted(aics.items(), key=lambda x:x[1]):
  print(val)

('None', [102773.75332957253])
('High', [103439.54001957676])
('Downtown', [104375.47924993062])
('Weekday', [110175.78819794345])
('ATMs', [111989.15389449417])
('Shops', [112424.65910466018])
('Center', [115218.43711395137])


# Interaction Effects Selection by Significance

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
X_train = ATM_train[ATM_train.columns.drop('Withdraw')]
y_train = ATM_train[['Withdraw']]
poly = PolynomialFeatures(3)
output_nparray = poly.fit_transform(X_train)
dc = poly.get_feature_names(X_train.columns)
X_train = pd.DataFrame(output_nparray, columns = dc)
X_train = X_train.drop('1', axis = 1)
X_train.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Shops,ATMs,Downtown,Weekday,Center,High,Shops^2,Shops ATMs,Shops Downtown,Shops Weekday,...,Weekday^3,Weekday^2 Center,Weekday^2 High,Weekday Center^2,Weekday Center High,Weekday High^2,Center^3,Center^2 High,Center High^2,High^3
0,10.18,10.0,1.0,0.0,0.0,0.0,103.6324,101.80,10.18,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9.74,10.0,1.0,1.0,0.0,0.0,94.8676,97.40,9.74,9.74,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.96,2.0,0.0,0.0,0.0,1.0,0.9216,1.92,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,9.58,9.0,1.0,1.0,0.0,1.0,91.7764,86.22,9.58,9.58,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1.03,4.0,0.0,1.0,0.0,1.0,1.0609,4.12,0.00,1.03,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
import statsmodels.api as sm
x_with_intercept = sm.add_constant(X_train, prepend=True)
model = sm.OLS(y_train, x_with_intercept)
results = model.fit()  # perform linear regression

print(results.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:               Withdraw   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.016e+06
Date:                Sat, 12 Nov 2022   Prob (F-statistic):               0.00
Time:                        14:08:14   Log-Likelihood:                -15849.
No. Observations:               22000   AIC:                         3.181e+04
Df Residuals:                   21944   BIC:                         3.226e+04
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [ ]:
X_train = X_train[['Shops', 'ATMs', 'Downtown', 'Weekday', 'Center','High','Shops Center','Shops Weekday','Weekday Center','Shops Weekday Center']]
X_train = sm.add_constant(X_train, prepend=True)
model = sm.OLS(y_train, X_train)
results = model.fit()  # perform linear regression

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               Withdraw   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 4.460e+06
Date:                Sat, 12 Nov 2022   Prob (F-statistic):               0.00
Time:                        14:08:14   Log-Likelihood:                -18348.
No. Observations:               22000   AIC:                         3.672e+04
Df Residuals:                   21989   BIC:                         3.681e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    9.7600 

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


# Linear Regression

In [ ]:
#Read the training dataset into a dataframe, and drop null values
ATM_train = pd.read_csv("ATM_training.csv")
ATM_train = ATM_train.dropna()
ATM_train.head()

,Shops,ATMs,Downtown,Weekday,Center,High,Withdraw
0,10.18,10,1,0,0,0,72.750556
1,9.74,10,1,1,0,0,66.720482
2,0.96,2,0,0,0,1,19.189516
3,9.58,9,1,1,0,1,67.388669
4,1.03,4,0,1,0,1,15.813127


In [ ]:
#Read the testing dataset into a dataframe, and drop null values
ATM_test = pd.read_csv("ATM_test.csv")
ATM_test = ATM_test.dropna()
ATM_test.head()

,Shops,ATMs,Downtown,Weekday,Center,High,Withdraw
0,0.94,4,0,0,0,0,15.876301
1,10.05,8,1,1,0,0,70.836720
2,10.06,13,1,1,0,0,67.735446
3,10.56,12,1,1,0,0,73.686420
4,10.17,7,1,1,0,0,74.524830


In [ ]:
ATM_train['Shops:Center'] = ATM_train['Shops']*ATM_train['Center']
ATM_train['Shops:Weekday'] = ATM_train['Shops']*ATM_train['Weekday']
ATM_train['Weekday:Center'] = ATM_train['Weekday']*ATM_train['Center']
ATM_train['Shops:Weekday:Center'] = ATM_train['Shops']*ATM_train['Weekday']*ATM_train['Center']
ATM_train.head()

,Shops,ATMs,Downtown,Weekday,Center,High,Withdraw,Shops:Center,Shops:Weekday,Weekday:Center,Shops:Weekday:Center
0,10.18,10,1,0,0,0,72.750556,0.0,0.00,0,0.0
1,9.74,10,1,1,0,0,66.720482,0.0,9.74,0,0.0
2,0.96,2,0,0,0,1,19.189516,0.0,0.00,0,0.0
3,9.58,9,1,1,0,1,67.388669,0.0,9.58,0,0.0
4,1.03,4,0,1,0,1,15.813127,0.0,1.03,0,0.0


In [ ]:
ATM_test['Shops:Center'] = ATM_test['Shops']*ATM_test['Center']
ATM_test['Shops:Weekday'] = ATM_test['Shops']*ATM_test['Weekday']
ATM_test['Weekday:Center'] = ATM_test['Weekday']*ATM_test['Center']
ATM_test['Shops:Weekday:Center'] = ATM_test['Shops']*ATM_test['Weekday']*ATM_test['Center']
ATM_test.head()

,Shops,ATMs,Downtown,Weekday,Center,High,Withdraw,Shops:Center,Shops:Weekday,Weekday:Center,Shops:Weekday:Center
0,0.94,4,0,0,0,0,15.876301,0.0,0.00,0,0.0
1,10.05,8,1,1,0,0,70.836720,0.0,10.05,0,0.0
2,10.06,13,1,1,0,0,67.735446,0.0,10.06,0,0.0
3,10.56,12,1,1,0,0,73.686420,0.0,10.56,0,0.0
4,10.17,7,1,1,0,0,74.524830,0.0,10.17,0,0.0


In [ ]:
#Define the withdraw variable as the response variable and the rest as the independent variables
response = ['Withdraw']
predictors = [x for x in list(ATM_train.columns) if x not in response]
X_train = ATM_train[predictors]
y_train = ATM_train[response]
X_test = ATM_test[predictors]
y_test = ATM_test[response]

In [ ]:
corr = X_train.corr()
print(corr)

                         Shops      ATMs  Downtown   Weekday    Center  \
Shops                 1.000000  0.872903  0.999131  0.013014  0.000004   
ATMs                  0.872903  1.000000  0.873726  0.009766 -0.003306   
Downtown              0.999131  0.873726  1.000000  0.012664 -0.000101   
Weekday               0.013014  0.009766  0.012664  1.000000 -0.007153   
Center                0.000004 -0.003306 -0.000101 -0.007153  1.000000   
High                  0.001820 -0.002616  0.001782 -0.006793  0.010521   
Shops:Center          0.163617  0.138746  0.163343 -0.001678  0.859604   
Shops:Weekday         0.617190  0.535823  0.616373  0.691636 -0.003045   
Weekday:Center        0.004895  0.002645  0.004694  0.176490  0.825553   
Shops:Weekday:Center  0.137070  0.117357  0.136751  0.153040  0.715861   

                          High  Shops:Center  Shops:Weekday  Weekday:Center  \
Shops                 0.001820      0.163617       0.617190        0.004895   
ATMs                 -0.002

In [ ]:
#Inspecting the relationship between each pair of variables
#sns.pairplot(X_train)

In [ ]:
X_train = ATM_train[['Shops', 'ATMs', 'Downtown', 'Weekday', 'Center','High']]

In [ ]:
formula = 'Withdraw ~ Shops + ATMs + Downtown  + Weekday + Center + High + Shops*Center + Shops*Weekday + Weekday*Center + Shops*Weekday*Center'

model = smf.ols(formula, ATM_train)

results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               Withdraw   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 4.460e+06
Date:                Sat, 12 Nov 2022   Prob (F-statistic):               0.00
Time:                        14:08:15   Log-Likelihood:                -18348.
No. Observations:               22000   AIC:                         3.672e+04
Df Residuals:                   21989   BIC:                         3.681e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                9.7600 

In [ ]:
y_pred = results.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(ATM_test['Withdraw'], y_pred)

0.3089239792894667

# Lasso Regression

Before Lasso Regression, Standardize all Predictors

In [ ]:
response = ['Withdraw']
numerical_predictors=['Shops', 'ATMs', 'Shops:Center', 'Shops:Weekday', 'Shops:Weekday:Center']
predictors=[x for x in list(ATM_train.columns) if x not in response] # the list all predictors
mu=ATM_train[numerical_predictors].mean() # mean for each feature
sigma=ATM_train[numerical_predictors].std() # std for each feature

In [ ]:
ATM_test[predictors].head()

,Shops,ATMs,Downtown,Weekday,Center,High,Shops:Center,Shops:Weekday,Weekday:Center,Shops:Weekday:Center
0,0.94,4,0,0,0,0,0.0,0.00,0,0.0
1,10.05,8,1,1,0,0,0.0,10.05,0,0.0
2,10.06,13,1,1,0,0,0.0,10.06,0,0.0
3,10.56,12,1,1,0,0,0.0,10.56,0,0.0
4,10.17,7,1,1,0,0,0.0,10.17,0,0.0


In [ ]:
ATM_train[numerical_predictors]=(ATM_train[numerical_predictors]-mu)/sigma
ATM_test[numerical_predictors]=(ATM_test[numerical_predictors]-mu)/sigma

In [ ]:
X_test = ATM_test[predictors]

In [ ]:
from sklearn.linear_model import LassoCV

lasso = LassoCV(cv=10)
lasso.fit(ATM_train[predictors], np.ravel(ATM_train[response]))

LassoCV(cv=10)

In [ ]:
print("LASSO Lambda: {0}".format(lasso.alpha_))

LASSO Lambda: 0.02474215837362939


In [ ]:
pd.DataFrame(lasso.coef_.round(3),index = predictors).T

,Shops,ATMs,Downtown,Weekday,Center,High,Shops:Center,Shops:Weekday,Weekday:Center,Shops:Weekday:Center
0,27.588,-3.592,-0.0,-1.919,2.094,0.885,5.265,-0.0,0.0,-4.266


In [ ]:
y_pred = lasso.predict(X_test)
test_error = mean_squared_error(ATM_test['Withdraw'], y_pred).round(4)
print(test_error)

0.9816


# Ridge Regression

In [ ]:
from sklearn.linear_model import RidgeCV

alphas = np.exp(np.linspace(-10,20,500)) 
ridge_cv = RidgeCV(alphas=alphas, cv=5)
ridge_cv.fit(ATM_train[predictors], np.ravel(ATM_train[response]))

RidgeCV(alphas=array([4.53999298e-05, 4.82131015e-05, 5.12005892e-05, 5.43731942e-05,
       5.77423873e-05, 6.13203499e-05, 6.51200182e-05, 6.91551300e-05,
       7.34402743e-05, 7.79909444e-05, 8.28235932e-05, 8.79556934e-05,
       9.34058002e-05, 9.91936187e-05, 1.05340075e-04, 1.11867391e-04,
       1.18799168e-04, 1.26160467e-04, 1.33977903e-04, 1.42279741e-04,
       1.51095996e-04, 1.60458543e-0...
       1.21720071e+08, 1.29262362e+08, 1.37272005e+08, 1.45777959e+08,
       1.54810978e+08, 1.64403721e+08, 1.74590870e+08, 1.85409259e+08,
       1.96898001e+08, 2.09098634e+08, 2.22055270e+08, 2.35814754e+08,
       2.50426834e+08, 2.65944340e+08, 2.82423376e+08, 2.99923523e+08,
       3.18508053e+08, 3.38244159e+08, 3.59203198e+08, 3.81460948e+08,
       4.05097881e+08, 4.30199460e+08, 4.56856438e+08, 4.85165195e+08]),
        cv=5)

In [ ]:
print("Ridge Lambda: {0}".format(ridge_cv.alpha_))

Ridge Lambda: 0.0005340238055689655


In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=ridge_cv.alpha_)
ridge.fit(ATM_train[predictors], np.ravel(ATM_train[response]))

Ridge(alpha=0.0005340238055689655)

In [ ]:
pd.DataFrame(ridge.coef_.round(3),index = predictors).T

,Shops,ATMs,Downtown,Weekday,Center,High,Shops:Center,Shops:Weekday,Weekday:Center,Shops:Weekday:Center
0,44.34,-3.675,-36.384,-2.023,2.991,0.999,5.156,0.013,-0.034,-4.394


In [ ]:
y_pred = ridge.predict(X_test)
test_error = mean_squared_error(ATM_test['Withdraw'], y_pred).round(4)
print(test_error)

0.3089


# Elastic Net CV

In [ ]:
from sklearn.linear_model import ElasticNetCV
enet_cv = ElasticNetCV(l1_ratio=[0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99], cv=5)
enet_cv.fit(ATM_train[predictors],np.ravel(ATM_train[response]))

ElasticNetCV(cv=5,
             l1_ratio=[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,
                       0.95, 0.99])

In [ ]:
y_pred = enet_cv.predict(X_test)
test_error = mean_squared_error(ATM_test['Withdraw'], y_pred).round(4)
print(test_error)

0.9858


In [ ]:
from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=enet_cv.alpha_, l1_ratio=enet_cv.l1_ratio_)
enet.fit(ATM_train[predictors],np.ravel(ATM_train[response]))

ElasticNet(alpha=0.024992079165282215, l1_ratio=0.99)

In [ ]:
y_pred = enet.predict(X_test)
test_error = mean_squared_error(ATM_test['Withdraw'], y_pred).round(4)
print(test_error)

0.9858


# KNN Classifiers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
kf=KFold(10) 

In [ ]:
ATM_train = pd.read_csv("ATM_training.csv")
ATM_train = ATM_train.dropna()

In [ ]:
ATM_test = pd.read_csv("ATM_test.csv")
ATM_test = ATM_train.dropna()

In [ ]:
def knn_test(predictors, response):
    
    neighbours=np.arange(1, 51)
    best_score = -np.inf
    
    for k in neighbours: 
        knn = KNeighborsRegressor(n_neighbors = k, metric='mahalanobis', metric_params={'V': ATM_train[predictors].cov()}) 
        scores = cross_val_score(knn, ATM_train[predictors], ATM_train[response], cv=10, scoring = 'neg_mean_squared_error')
        # taking the average of scores across 10 folds
        cv_score = np.mean(scores)
        # use the cv score for model selection
        if cv_score >= best_score:
            best_score = cv_score
            best_knn = knn
    
    knn = best_knn
    # train the selected model with the whole train set
    knn.fit(ATM_train[predictors], ATM_train[response])
    # Predict the test data with the selected and re-estimated model
    predictions = knn.predict(ATM_test[predictors])
    test_mse = mean_squared_error(ATM_test[response], predictions)
    cv_mse= -best_score
    print('Chosen K: {}'.format(knn.n_neighbors))
    
    return test_mse, cv_mse

In [ ]:
test_error = knn_test(['Shops', 'ATMs', 'Downtown', 'Weekday', 'Center','High'], 'Withdraw')[0]

Chosen K: 7


In [ ]:
print(test_error)

0.25022989535916096


# Neural Networks/Deep Learning

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# from tensorflow import keras 
from keras.layers.core import Dense 
from keras.models import Sequential


In [ ]:
np.random.seed(1)
import tensorflow as tf
tf.random.set_seed(0)

In [ ]:
ATM_train = pd.read_csv('ATM_training.csv')
ATM_train.dropna()

ATM_test = pd.read_csv('ATM_test.csv')
ATM_test.dropna()

,Shops,ATMs,Downtown,Weekday,Center,High,Withdraw
0,0.94,4,0,0,0,0,15.876301
1,10.05,8,1,1,0,0,70.836720
2,10.06,13,1,1,0,0,67.735446
3,10.56,12,1,1,0,0,73.686420
4,10.17,7,1,1,0,0,74.524830
...,...,...,...,...,...,...,...
145,9.58,12,1,0,0,0,64.871203
146,0.98,1,0,1,0,0,17.918977
147,9.98,8,1,1,0,1,72.159322
148,9.58,9,1,1,0,0,65.373738


In [ ]:
ATM_train['Shops:Center'] = ATM_train['Shops']*ATM_train['Center']
ATM_train['Shops:Weekday'] = ATM_train['Shops']*ATM_train['Weekday']
ATM_train['Weekday:Center'] = ATM_train['Weekday']*ATM_train['Center']
ATM_train['Shops:Weekday:Center'] = ATM_train['Shops']*ATM_train['Weekday']*ATM_train['Center']
ATM_train.head()

,Shops,ATMs,Downtown,Weekday,Center,High,Withdraw,Shops:Center,Shops:Weekday,Weekday:Center,Shops:Weekday:Center
0,10.18,10,1,0,0,0,72.750556,0.0,0.00,0,0.0
1,9.74,10,1,1,0,0,66.720482,0.0,9.74,0,0.0
2,0.96,2,0,0,0,1,19.189516,0.0,0.00,0,0.0
3,9.58,9,1,1,0,1,67.388669,0.0,9.58,0,0.0
4,1.03,4,0,1,0,1,15.813127,0.0,1.03,0,0.0


In [ ]:
ATM_test['Shops:Center'] = ATM_test['Shops']*ATM_test['Center']
ATM_test['Shops:Weekday'] = ATM_test['Shops']*ATM_test['Weekday']
ATM_test['Weekday:Center'] = ATM_test['Weekday']*ATM_test['Center']
ATM_test['Shops:Weekday:Center'] = ATM_test['Shops']*ATM_test['Weekday']*ATM_test['Center']
ATM_test.head()

,Shops,ATMs,Downtown,Weekday,Center,High,Withdraw,Shops:Center,Shops:Weekday,Weekday:Center,Shops:Weekday:Center
0,0.94,4,0,0,0,0,15.876301,0.0,0.00,0,0.0
1,10.05,8,1,1,0,0,70.836720,0.0,10.05,0,0.0
2,10.06,13,1,1,0,0,67.735446,0.0,10.06,0,0.0
3,10.56,12,1,1,0,0,73.686420,0.0,10.56,0,0.0
4,10.17,7,1,1,0,0,74.524830,0.0,10.17,0,0.0


In [ ]:
#Define the withdraw variable as the response variable and the rest as the independent variables
response = ['Withdraw']
predictors = [x for x in list(ATM_train.columns) if x not in response]
X_train = ATM_train[predictors]
y_train = ATM_train[response]
X_test = ATM_test[predictors]
y_test = ATM_test[response]

In [ ]:
print(ATM_train.shape, ATM_test.shape)

(22000, 11) (150, 11)


**Preprocessing**

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fitting the scaler
ATM_train_fit = scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

We test the model using a variety of Epochs from a the range of 60 to 100. Because the computation time for this is very long, after the initial run, the model was exported using joblib so it can be called upon the next run. The code used to test the list of Epochs is at the very bottom of this notebook

# BEST NN MODEL FOUND
---

THE CODE BELOW IS A PRECOMPUTED MODEL (SAVED USING JOBLIB) USING THE SCRIPT AT THE VERY BOTTOM OF THE NOTEBOOK THAT WAS RUN >10 TIMES TO ACCOUNT FOR STOCHASTIC RANDOMNESS

In [ ]:
import keras
## load tensorflow model
save_path = './Group_32_Best_NN_Model.h5'
loaded_model = keras.models.load_model(save_path)
test_error = loaded_model.evaluate(X_test, y_test)
print(round(test_error, 4))

5/5 [==============================] - 0s 3ms/step - loss: 0.2275
0.2275


---

From the code at the very bottom of this notebook, epochs = 70 was found to be the best for lowest Test Error, so let's give it a run first. Note, results will vary each run due to the stochastic nature of NNs

In [ ]:
model = Sequential()
model.add(Dense(11, input_dim=10, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='MSE', optimizer='adam')
model.fit(X_train, y_train, epochs = 70, batch_size = 10)
test_error = model.evaluate(X_test, y_test)
print('\n MSE of test data using adam optimizer: ')
print(round(test_error, 4))

Epoch 1/70
2200/2200 [==============================] - 4s 2ms/step - loss: 1363.4503
Epoch 2/70
2200/2200 [==============================] - 4s 2ms/step - loss: 20.9554
Epoch 3/70
2200/2200 [==============================] - 4s 2ms/step - loss: 1.6193
Epoch 4/70
2200/2200 [==============================] - 4s 2ms/step - loss: 0.9851
Epoch 5/70
2200/2200 [==============================] - 4s 2ms/step - loss: 0.7239
Epoch 6/70
2200/2200 [==============================] - 4s 2ms/step - loss: 0.5587
Epoch 7/70
2200/2200 [==============================] - 4s 2ms/step - loss: 0.4576
Epoch 8/70
2200/2200 [==============================] - 4s 2ms/step - loss: 0.3901
Epoch 9/70
2200/2200 [==============================] - 4s 2ms/step - loss: 0.3558
Epoch 10/70
2200/2200 [==============================] - 4s 2ms/step - loss: 0.3320
Epoch 11/70
2200/2200 [==============================] - 4s 2ms/step - loss: 0.3213
Epoch 12/70
2200/2200 [==============================] - 4s 2ms/step - loss: 0.31

Try NN with the Stochastic gradient descent (SGD) optimizer

In [ ]:
model = Sequential()
model.add(Dense(11, input_dim=10, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='MSE', optimizer='SGD')
model.fit(X_train, y_train, epochs = 70, batch_size = 10)
test_error = model.evaluate(X_test, y_test)
print('\n MSE of test data using sgd optimizer: ')
print(round(test_error, 4))

Epoch 1/70
2200/2200 [==============================] - 4s 1ms/step - loss: 132.7096
Epoch 2/70
2200/2200 [==============================] - 3s 1ms/step - loss: 116.3523
Epoch 3/70
2200/2200 [==============================] - 3s 1ms/step - loss: 104.7797
Epoch 4/70
2200/2200 [==============================] - 3s 1ms/step - loss: 104.4519
Epoch 5/70
2200/2200 [==============================] - 3s 1ms/step - loss: 104.4017
Epoch 6/70
2200/2200 [==============================] - 3s 1ms/step - loss: 104.2700
Epoch 7/70
2200/2200 [==============================] - 3s 1ms/step - loss: 104.3302
Epoch 8/70
2200/2200 [==============================] - 3s 1ms/step - loss: 104.1663
Epoch 9/70
2200/2200 [==============================] - 3s 1ms/step - loss: 104.2611
Epoch 10/70
2200/2200 [==============================] - 3s 1ms/step - loss: 104.2640
Epoch 11/70
2200/2200 [==============================] - 3s 1ms/step - loss: 104.2721
Epoch 12/70
2200/2200 [==============================] - 3s 1ms

Test epochs from a range of 60 to 80, incremented by 1.

In [ ]:
import keras
#Create an array of epochs to test
epoch_list = range(60,80,1)
#Set up the NN
model = Sequential()
model.add(Dense(11, input_dim=10, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='MSE', optimizer='adam')
#Compute the best test error
test_error = np.inf
for epoch in epoch_list:
  model.fit(X_train, y_train, epochs = epoch, batch_size = 10, verbose = 0)
  error = model.evaluate(X_test, y_test)
  if error < test_error:
    test_error = error
    best_model = model
    best_epoch = epoch
  print(f"Epoch {epoch}: complete, Test Error: {error}")
#Export the best model, so we don't have to run the code again
save_path = './Group_32_Best_NN_Model.h5'
best_model.save(save_path)
print("------------------------------------------")
print("           FINAL RESULTS                  ")
print("------------------------------------------")
print('\n MSE of test data using adam optimizer:  ')
print('\nBest epoch no.: ', best_epoch)
print('\n MSE of test data using adam optimizer:  ')
print(round(test_error, 4))

5/5 [==============================] - 0s 3ms/step - loss: 0.2526
Epoch 60: complete, Test Error: 0.25258469581604004
5/5 [==============================] - 0s 3ms/step - loss: 0.2388
Epoch 61: complete, Test Error: 0.2388327717781067
5/5 [==============================] - 0s 3ms/step - loss: 0.2295
Epoch 62: complete, Test Error: 0.22949358820915222
5/5 [==============================] - 0s 3ms/step - loss: 0.2489
Epoch 63: complete, Test Error: 0.24885815382003784
5/5 [==============================] - 0s 3ms/step - loss: 0.2369
Epoch 64: complete, Test Error: 0.23693063855171204
5/5 [==============================] - 0s 3ms/step - loss: 0.2421
Epoch 65: complete, Test Error: 0.2420942187309265
5/5 [==============================] - 0s 3ms/step - loss: 0.2370
Epoch 66: complete, Test Error: 0.2370048314332962
5/5 [==============================] - 0s 3ms/step - loss: 0.2657
Epoch 67: complete, Test Error: 0.2657413184642792
5/5 [==============================] - 0s 3ms/step - loss: 0

Because epochs = 70 was found to be the best model optimizer, the following code is a very inefficient way of getting a potentially even more accurate result by running epochs = 70 over many repetitive iterations and selecting the best score.

In [ ]:
import keras
#Create an array of epochs to test
iterations = range(1,30,1)
#Set up the NN
model = Sequential()
model.add(Dense(11, input_dim=10, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='MSE', optimizer='adam')
#Compute the best test error
test_error = np.inf
for epoch in iterations:
  model.fit(X_train, y_train, epochs = 70, batch_size = 10, verbose = 0)
  error = model.evaluate(X_test, y_test)
  if error < test_error:
    test_error = error
    best_model = model
    best_epoch = epoch
  print(f"Epoch {epoch}: complete, Test Error: {error}")
#Export the best model, so we don't have to run the code again
save_path = './Group_32_Best_NN_Model.h5'
best_model.save(save_path)
print("------------------------------------------")
print("           FINAL RESULTS                  ")
print("------------------------------------------")
print('\n MSE of test data using adam optimizer:  ')
print('\nBest epoch no.: ', best_epoch)
print('\n MSE of test data using adam optimizer:  ')
print(round(test_error, 4))